In [34]:
class RBNode:
    def __init__(self, key, color='R', left=None, right=None, parent=None):
        self.key = key
        self.color = color
        self.left = left
        self.right = right
        self.parent = parent


class RedBlackTree:
    def __init__(self):
        self.nil = RBNode(None, 'B')  # Sentinel node
        self.root = self.nil

    def rb_insert(self, key):
        new_node = RBNode(key)
        y = self.nil
        x = self.root

        while x != self.nil:
            y = x
            if new_node.key < x.key:
                x = x.left
            else:
                x = x.right

        new_node.parent = y
        if y == self.nil:
            self.root = new_node
        elif new_node.key < y.key:
            y.left = new_node
        else:
            y.right = new_node

        new_node.left = self.nil
        new_node.right = self.nil
        new_node.color = 'R'

        self.rb_insert_fixup(new_node)

    def left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.nil:
            y.left.parent = x
        y.parent = x.parent
        if x.parent == self.nil:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def right_rotate(self, y):
        x = y.left
        y.left = x.right
        if x.right != self.nil:
            x.right.parent = y
        x.parent = y.parent
        if y.parent == self.nil:
            self.root = x
        elif y == y.parent.left:
            y.parent.left = x
        else:
            y.parent.right = x
        x.right = y
        y.parent = x

    def rb_insert_fixup(self, z):
        while z.parent.color == 'R':
            if z.parent == z.parent.parent.left:
                y = z.parent.parent.right
                if y.color == 'R':
                    z.parent.color = 'B'
                    y.color = 'B'
                    z.parent.parent.color = 'R'
                    z = z.parent.parent
                else:
                    if z == z.parent.right:
                        z = z.parent
                        self.left_rotate(z)
                    z.parent.color = 'B'
                    z.parent.parent.color = 'R'
                    self.right_rotate(z.parent.parent)
            else:
                y = z.parent.parent.left
                if y.color == 'R':
                    z.parent.color = 'B'
                    y.color = 'B'
                    z.parent.parent.color = 'R'
                    z = z.parent.parent
                else:
                    if z == z.parent.left:
                        z = z.parent
                        self.right_rotate(z)
                    z.parent.color = 'B'
                    z.parent.parent.color = 'R'
                    self.left_rotate(z.parent.parent)
        self.root.color = 'B'

    def rb_transplant(self, u, v):
        if u.parent == self.nil:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    def find_node(self, key, node=None):
        if node is None:
            node = self.root
        while node != self.nil and node.key != key:
            if key < node.key:
                node = node.left
            else:
                node = node.right
        return node

    def rb_delete(self, z):
        if z == self.nil:
            return  # Ignore deletion for nil node

        y = z
        y_original_color = y.color
        if z.left == self.nil:
            x = z.right
            self.rb_transplant(z, z.right)
        elif z.right == self.nil:
            x = z.left
            self.rb_transplant(z, z.left)
        else:
            y = self.tree_minimum(z.right)
            y_original_color = y.color
            x = y.right
            if y.parent == z:
                x.parent = y
            else:
                self.rb_transplant(y, y.right)
                y.right = z.right
                y.right.parent = y
            self.rb_transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color
        if y_original_color == 'B':
            self.rb_delete_fixup(x)

    def rb_delete_fixup(self, x):
        while x != self.root and x.color == 'B':
            if x == x.parent.left:
                w = x.parent.right
                if w.color == 'R':
                    w.color = 'B'
                    x.parent.color = 'R'
                    self.left_rotate(x.parent)
                    w = x.parent.right
                if w.left.color == 'B' and w.right.color == 'B':
                    w.color = 'R'
                    x = x.parent
                else:
                    if w.right.color == 'B':
                        w.left.color = 'B'
                        w.color = 'R'
                        self.right_rotate(w)
                        w = x.parent.right
                    w.color = x.parent.color
                    x.parent.color = 'B'
                    w.right.color = 'B'
                    self.left_rotate(x.parent)
                    x = self.root
            else:
                w = x.parent.left
                if w.color == 'R':
                    w.color = 'B'
                    x.parent.color = 'R'
                    self.right_rotate(x.parent)
                    w = x.parent.left
                if w.right.color == 'B' and w.left.color == 'B':
                    w.color = 'R'
                    x = x.parent
                else:
                    if w.left.color == 'B':
                        w.right.color = 'B'
                        w.color = 'R'
                        self.left_rotate(w)
                        w = x.parent.left
                    w.color = x.parent.color
                    x.parent.color = 'B'
                    w.left.color = 'B'
                    self.right_rotate(x.parent)
                    x = self.root
        x.color = 'B'

    def print_tree(self):
        self.print_tree_recursive(self.root, "", True)

    def print_tree_recursive(self, node, indent, last):
        if node != self.nil:
            color = "R" if node.color == 'R' else "B"
            print(indent, end="")
            if last:
                print("└── ", end="")
                indent += "    "
            else:
                print("├── ", end="")
                indent += "│   "
            print(f"{node.key} ({color})")
            self.print_tree_recursive(node.left, indent, False)
            self.print_tree_recursive(node.right, indent, True)

    def tree_minimum(self, x):
        while x != self.nil and x.left != self.nil:
            x = x.left
        return x

    def tree_maximum(self, x):
        while x != self.nil and x.right != self.nil:
            x = x.right
        return x

    def find_successor(self, node):
        if node.right != self.nil:  # If the node has a right subtree
            return self.tree_minimum(node.right)
        # Otherwise, move up to the parent until we come from the left
        y = node.parent
        while y != self.nil and node == y.right:
            node = y
            y = y.parent
        return y

    def find_predecessor(self, node):
        if node.left != self.nil:  # If the node has a left subtree
            return self.tree_maximum(node.left)
        # Otherwise, move up to the parent until we come from the right
        y = node.parent
        while y != self.nil and node == y.left:
            node = y
            y = y.parent
        return y


In [35]:
rbt = RedBlackTree()
rbt.print_tree()
# a)
keys_to_insert = [15, 10, 41, 20, 16, 35, 28, 17, 14, 39, 30, 3, 7, 12, 38, 26, 19, 47, 21, 23]
for key in keys_to_insert:
    rbt.rb_insert(key)
    rbt.print_tree()
    print("\n")

└── 15 (B)


└── 15 (B)
    ├── 10 (R)


└── 15 (B)
    ├── 10 (R)
    └── 41 (R)


└── 15 (B)
    ├── 10 (B)
    └── 41 (B)
        ├── 20 (R)


└── 15 (B)
    ├── 10 (B)
    └── 20 (B)
        ├── 16 (R)
        └── 41 (R)


└── 15 (B)
    ├── 10 (B)
    └── 20 (R)
        ├── 16 (B)
        └── 41 (B)
            ├── 35 (R)


└── 15 (B)
    ├── 10 (B)
    └── 20 (R)
        ├── 16 (B)
        └── 35 (B)
            ├── 28 (R)
            └── 41 (R)


└── 15 (B)
    ├── 10 (B)
    └── 20 (R)
        ├── 16 (B)
        │   └── 17 (R)
        └── 35 (B)
            ├── 28 (R)
            └── 41 (R)


└── 15 (B)
    ├── 10 (B)
    │   └── 14 (R)
    └── 20 (R)
        ├── 16 (B)
        │   └── 17 (R)
        └── 35 (B)
            ├── 28 (R)
            └── 41 (R)


└── 20 (B)
    ├── 15 (R)
    │   ├── 10 (B)
    │   │   └── 14 (R)
    │   └── 16 (B)
    │       └── 17 (R)
    └── 35 (R)
        ├── 28 (B)
        └── 41 (B)
            ├── 39 (R)


└── 20 (B)
    ├── 15 (R)
    │   ├

In [36]:
# b)
keys_to_delete = [35, 17, 41]

for key in keys_to_delete:
    node_to_delete = rbt.find_node(key)
    rbt.rb_delete(node_to_delete)
    rbt.print_tree()

rbt.rb_delete(rbt.root)
rbt.print_tree()

└── 20 (B)
    ├── 15 (B)
    │   ├── 10 (R)
    │   │   ├── 3 (B)
    │   │   │   └── 7 (R)
    │   │   └── 14 (B)
    │   │       ├── 12 (R)
    │   └── 17 (B)
    │       ├── 16 (R)
    │       └── 19 (R)
    └── 38 (B)
        ├── 28 (R)
        │   ├── 23 (B)
        │   │   ├── 21 (R)
        │   │   └── 26 (R)
        │   └── 30 (B)
        └── 41 (R)
            ├── 39 (B)
            └── 47 (B)
└── 20 (B)
    ├── 15 (B)
    │   ├── 10 (R)
    │   │   ├── 3 (B)
    │   │   │   └── 7 (R)
    │   │   └── 14 (B)
    │   │       ├── 12 (R)
    │   └── 19 (B)
    │       ├── 16 (R)
    └── 38 (B)
        ├── 28 (R)
        │   ├── 23 (B)
        │   │   ├── 21 (R)
        │   │   └── 26 (R)
        │   └── 30 (B)
        └── 41 (R)
            ├── 39 (B)
            └── 47 (B)
└── 20 (B)
    ├── 15 (B)
    │   ├── 10 (R)
    │   │   ├── 3 (B)
    │   │   │   └── 7 (R)
    │   │   └── 14 (B)
    │   │       ├── 12 (R)
    │   └── 19 (B)
    │       ├── 16 (R)
    └── 38 (B)
        ├

In [38]:
print(rbt.find_successor(rbt.root).key)
print(rbt.find_predecessor(rbt.root).key)

23
19
